In [2]:
# Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Hyperparameters
n_hidden = 35
lr = 0.01
epochs = 1000

string = 'hello pytorch. how long can a rnn cell remember?'
chars = 'abcdefghijklmnopqrstuvwxyz ?!.,:;01'
char_list = [i for i in chars]
n_letters = len(char_list)


In [9]:
# 문자열을 onehot벡터의 스택으로 만드는 함수
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)   # 문자가 몇번재 문자인지 찾기
        zero = np.zeros(shape=n_letters, dtype=int)   # 0으로만 구성된 배열만들기
        zero[idx] = 1   # 해당 문자 인덱스만 1로 바꿔주기
        start = np.vstack([start, zero])
        # start와 새로생긴 zero를 붙이고 이를 start에 할당
        # 이게 반복되면 start에는 문자를 onehot벡터로 바꾼 배열들이 쌓임
    output = np.vstack([start, zero])
    # 문자열이 다 끝나면 쌓아온 start와 end를 붙임
    return output

# np.zeros(5) = [0, 0, 0, 0, 0]

# onehot벡터를 문자로 바꾸는 함수
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)   # 텐서를 입력받아 넘파이 배열로 바꿈
    return char_list[onehot.argmax()]    # onehot벡터의 최댓값(1) 위치 인덱스로 문자 찾음


In [10]:
# RNN 모델 클래스 구현
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input) + self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    # 아직 입력 없을때(t=0) hidden state(초기의 은닉층 값) 0으로 초기화
    
rnn = RNN(n_letters, n_hidden, n_letters)


In [11]:
# 손실함수의 최적화 함수
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [13]:
# Training
# 문자열을 onehot벡터로 변환한 넘파이 배열을 다시 토치 텐서 형태로 바꿈
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1, :]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        input_ = output
        
    total_loss.backward()
    optimizer.step()
    
    if i%10 == 0:
        print(total_loss)

        
start = torch.zeros(1, len(char_list))
start[:, -2] = 1
with torch.no_grad():
    hidden=  rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
        
print(output_string)

tensor(0.0025, grad_fn=<AddBackward0>)
tensor(0.0024, grad_fn=<AddBackward0>)
tensor(0.0071, grad_fn=<AddBackward0>)
tensor(0.0075, grad_fn=<AddBackward0>)
tensor(0.0042, grad_fn=<AddBackward0>)
tensor(0.0028, grad_fn=<AddBackward0>)
tensor(0.0026, grad_fn=<AddBackward0>)
tensor(0.0024, grad_fn=<AddBackward0>)
tensor(0.0023, grad_fn=<AddBackward0>)
tensor(0.0022, grad_fn=<AddBackward0>)
tensor(0.0022, grad_fn=<AddBackward0>)
tensor(0.0329, grad_fn=<AddBackward0>)
tensor(0.0050, grad_fn=<AddBackward0>)
tensor(0.0033, grad_fn=<AddBackward0>)
tensor(0.0025, grad_fn=<AddBackward0>)
tensor(0.0023, grad_fn=<AddBackward0>)
tensor(0.0021, grad_fn=<AddBackward0>)
tensor(0.0020, grad_fn=<AddBackward0>)
tensor(0.0019, grad_fn=<AddBackward0>)
tensor(0.0019, grad_fn=<AddBackward0>)
tensor(0.0019, grad_fn=<AddBackward0>)
tensor(0.0251, grad_fn=<AddBackward0>)
tensor(0.0045, grad_fn=<AddBackward0>)
tensor(0.0029, grad_fn=<AddBackward0>)
tensor(0.0022, grad_fn=<AddBackward0>)
tensor(0.0019, grad_fn=<A